## GraphBG-MS: SpectConv -> MetaCells -> Integration -> VB-GMM -> Propagation 

In [1]:
from numpy.random import default_rng
import scanpy as sc
# import squidpy as sq
from anndata import AnnData
import scipy
from sklearn.metrics.cluster import adjusted_rand_score
import numpy as np
import pandas as pd
import seaborn as sns
import os
import torch
import pandas as pd
from sklearn import metrics
import multiprocessing as mp
from sklearn.metrics.cluster import normalized_mutual_info_score
import pickle
from sklearn.neighbors import kneighbors_graph
# from sklearn.mixture import GaussianMixture
# from sklearn.mixture import BayesianGaussianMixture
import time
from utils import cKD_refine_label
sc.logging.print_header()

/home/vanhoan310/miniconda3/envs/gnn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.1 anndata==0.9.2 umap==0.5.7 numpy==1.24.4 scipy==1.8.1 pandas==1.4.2 scikit-learn==1.1.1 statsmodels==0.14.1 python-igraph==0.11.8 pynndescent==0.5.13


In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
# MERFISH data
data_dir = '/data/hoan/spatial_transcriptomics/data/SDMBench/Data/378k/BrainAgingSpatialAtlas_MERFISH.h5ad'
adata_big=sc.read_h5ad(data_dir)
donor_id = np.unique(adata_big.obs['donor_id'])
adata_big.obsm['spatial'] = adata_big.obsm['spatial_coords']
adata_big.obs['batch'] = adata_big.obs['donor_id'].astype(str) +'Slice'+ adata_big.obs['slice'].astype(str)
adata_big.obs['Region'] = adata_big.obs['tissue']

In [4]:
# import scanpy as sc, anndata as ad
# import glob
# from sklearn.decomposition import PCA
# # from GraphST.utils import refine_label
# from sklearn.preprocessing import StandardScaler
# # from GraphST.utils import mclust_R
# data_dir_full = []
# adata_list = []
# for data_dir in glob.glob("/data/hoan/spatial_transcriptomics/data/SDMBench/Data/MERFISH/*.h5ad", recursive=True):
#     # print(data_dir)
#     data_dir_full.append(data_dir)
#     ## Read and preprocessing data
#     adata=sc.read_h5ad(data_dir)
#     n_features = adata.shape[1]
#     if n_features > 5000:
#         sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000)
#     sc.pp.normalize_total(adata, target_sum=1e4)
#     sc.pp.log1p(adata)
#     sc.pp.scale(adata, zero_center=False, max_value=10)
#     if n_features > 5000:
#         adata =  adata[:, adata.var['highly_variable']]
#     sc.tl.pca(adata, n_comps=20)
#     adata_list.append(adata)

# adata_big = ad.concat(adata_list, join="inner")
# batch = [data_dir_full[i][-9:-5] for i in range(len(data_dir_full)) for j in range(adata_list[i].shape[0])]
# adata_big.obs['batch'] = batch
# data_dir_full

In [5]:
# adata_big.obsm['spatial']

In [6]:
GraphBG = 'domain'
adata_big.obs['GraphBG-MS_labels'] = 0

In [7]:
from sklearn.cluster import (KMeans, MiniBatchKMeans, AgglomerativeClustering, SpectralClustering, Birch)
from sklearn.neighbors import kneighbors_graph
# from GraphST.utils import refine_label

In [8]:
## Parallel implementation
from joblib import Parallel, delayed
def process_slice_donor(batch_id):
    # print(sliceID, donorID)
    adata = adata_big[adata_big.obs['batch'] == batch_id].copy()
    if adata.shape[0] > 10:
        n_clusters = 200 #100       
        # Clustering
        connectivity = kneighbors_graph(adata.obsm['spatial'], n_neighbors=4, include_self=False)
        connectivity = 0.5 * (connectivity + connectivity.T)
        embedding = connectivity.dot(adata.obsm['X_pca'])

        clusteringobj = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, tol=1e-3, max_iter=10000)
        adata.obs[GraphBG] = clusteringobj.fit_predict(embedding)
        # adata.obs[GraphBG] = refine_label(adata, radius=50, key=GraphBG)
        adata.obs[GraphBG] = cKD_refine_label(np.array(adata.obsm['spatial']), adata.obs[GraphBG], k = 45)
        cluster_vec = np.unique(adata.obs[GraphBG])
        clusterIDvec = []
        clusterValuevec = []
        cluster2Cellsvec = []
        for clusterID in cluster_vec:
        # for clusterID in range(n_clusters):
            sliceClusterID = batch_id + 'ClusterID' + str(clusterID)
            sliceClusterValue = np.mean(embedding[adata.obs[GraphBG] == clusterID], axis=0)
            clusterIDvec.append(sliceClusterID)
            clusterValuevec.append(sliceClusterValue)
            cluster2Cellsvec.append(adata.obs_names[adata.obs[GraphBG] == clusterID])
        return ((clusterIDvec, clusterValuevec, cluster2Cellsvec))

In [9]:
time_st = time.time()
# Parallel execution
batch_vec = np.unique(adata_big.obs['batch'])
# batch_vec = batch_vec[:5]
# n_jobs = 1 in the paper
results = Parallel(n_jobs=10)(delayed(process_slice_donor)(batch) for batch in batch_vec)
time_ed = time.time()
time_batch_clustering = time_ed-time_st
print(time_batch_clustering/60)

1.3866403341293334


In [10]:
sliceCluster2SampleID = {}
pseudoSamples_column = []
pseudoSamples_value = []
for i in range(len(results)):
    if results[i] is not None:
        x, y, z = results[i]
        pseudoSamples_column += x
        pseudoSamples_value += y
        for j in range(len(x)):
            sliceCluster2SampleID[x[j]] = z[j]

In [11]:
# pseudoSamples = pd.DataFrame(np.array(pseudoSamples_value)[:len(pseudoSamples_column),].T, columns = list(pseudoSamples_column))
pseudoSamples = pd.DataFrame(np.array(pseudoSamples_value).T, columns = list(pseudoSamples_column))

In [12]:
# pseudoSamples

In [13]:
n_clusters = len(set(adata_big.obs['Region']))
if np.nan in set(adata_big.obs['Region']):
    n_clusters = n_clusters-1 # remove nan

In [14]:
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score

In [15]:
from CustomVBG import CustomBayesianGaussianMixture
gbg = CustomBayesianGaussianMixture(n_components=n_clusters, covariance_type='tied', random_state=42, init_params = 'random_from_data', n_init = 15, max_iter = 1000)

In [16]:
pseudoSamples_list = pseudoSamples_column
if 'BrainAging' in pseudoSamples_list[0]:
    print('Set batch for Brain Aging Dataset')
    batch_labels = [item[0:30] for item in pseudoSamples_list] # For 
else:
    print('Set batch labels')
    batch_labels = [item[0:5] for item in pseudoSamples_list] # For

Set batch for Brain Aging Dataset


In [17]:
np.unique(batch_labels)

array(['MsBrainAgingSpatialDonor_10Sli', 'MsBrainAgingSpatialDonor_11Sli',
       'MsBrainAgingSpatialDonor_12Sli', 'MsBrainAgingSpatialDonor_1Slic',
       'MsBrainAgingSpatialDonor_2Slic', 'MsBrainAgingSpatialDonor_3Slic',
       'MsBrainAgingSpatialDonor_4Slic', 'MsBrainAgingSpatialDonor_5Slic',
       'MsBrainAgingSpatialDonor_6Slic', 'MsBrainAgingSpatialDonor_7Slic',
       'MsBrainAgingSpatialDonor_8Slic', 'MsBrainAgingSpatialDonor_9Slic'],
      dtype='<U30')

In [18]:
X = pseudoSamples.T.values
import scanpy.external as sce
adata = sc.AnnData(X)
adata.obsm['X_pca'] = X
adata.obs["batch"] = batch_labels
time_st = time.time()
sc.pp.combat(adata, key="batch", inplace=True)
# sce.pp.harmony_integrate(adata, key="batch", max_iter_harmony=40)
time_ed = time.time()
time_combat = time_ed-time_st
adata

AnnData object with n_obs × n_vars = 5285 × 50
    obs: 'batch'
    obsm: 'X_pca'

In [19]:
X = adata.X
if 'X_pca_harmony' in adata.obsm:
    print ("Use harmony")
    X  = adata.obsm['X_pca_harmony']
# Store clustering results
time_st = time.time()
multi_clusterLabels = gbg.fit_predict(X)
adata_big.obs['GraphBG-MS_labels'] = 0
for i in range(len(pseudoSamples_list)):
    pseudoSamplesX = pseudoSamples_list[i]
    adata_big.obs.loc[sliceCluster2SampleID[pseudoSamplesX], 'GraphBG-MS_labels'] = multi_clusterLabels[i]
#print(name, normalized_mutual_info_score(adata_big.obs['tissue'], adata_big.obs['GraphBG-MS_labels']))
print('Spatial correction per slice')
for batch in batch_vec:
    mask_ = (adata_big.obs['batch'] == batch)
    adata_big.obs.loc[mask_, 'GraphBG-MS_labels'] = cKD_refine_label(np.array(adata_big[mask_].obsm['spatial']),
                                                                    adata_big.obs.loc[mask_, 'GraphBG-MS_labels'], k = 45)
true_labels = pd.factorize(adata_big.obs['Region'])[0]

Spatial correction per slice


In [20]:
print(normalized_mutual_info_score(true_labels[true_labels>0], adata_big.obs['GraphBG-MS_labels'][true_labels>0]))

time_ed = time.time()
time_Kmeans = time_ed-time_st

0.7048365747052985
